# In this notebook we would train the GRU model

In [ ]:
keep 
atr_14  , OBV

drop

'ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal', 'trending_market'

In [ ]:
drop_gru = [
    'open', 'high', 'low', 'typical_price', 'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'bollinger_width', 'volatility_regime', 'CCI', 'stoch_%D',
    'parkinson_vol', 'ema_cross_down', 'macd_cross_down',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'trending_market',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6''ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal'
]

In [ ]:
"""
Precision-Optimized GRU for Bitcoin Direction Prediction
=======================================================
Modified to maximize PRECISION with 2x weight to precision over recall.
Using F0.5 score (beta=0.5) which emphasizes precision.

F-beta formula:
- F2 (β=2): Emphasizes RECALL (2x weight to recall)
- F1 (β=1): Equal weight to precision and recall  
- F0.5 (β=0.5): Emphasizes PRECISION (2x weight to precision)
"""

import os
import json
import optuna
import warnings
import joblib
import time
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score, accuracy_score
import tensorflow as tf
from tensorflow import keras

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.get_logger().setLevel('ERROR')

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ═══════════════════════════════════════════════════════════════════════
# Configuration - PRECISION FOCUSED
# ═══════════════════════════════════════════════════════════════════════

CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
VAL_FRAC = 0.2

# PRECISION OPTIMIZATION: Use F0.5 score (emphasizes precision)
BETA = 0.5  # ← CHANGED: β=0.5 gives 2x weight to PRECISION
PRECISION_WEIGHT = 2.0  # Additional precision weighting

MODEL_NAME = "gru_precision_optimized.h5"
N_TRIALS = 30
TIMEOUT = 100 * 60

# Same comprehensive drop list
DROP_COLS = [
    'open', 'high', 'low', 'close',
    'typical_price', 'vwap_24h', 'close_4h',
    'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'bollinger_upper', 'bollinger_lower', 'bollinger_width',
    'resistance_level', 'support_level',
    'high_low', 'high_close', 'low_close', 'true_range',
    'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'CCI', 'stoch_%D', 'parkinson_vol',
    'ema_cross_down', 'macd_cross_down', 'ema_cross_up', 'macd_cross_up',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'trending_market',
    'oversold_reversal', 'overbought_reversal',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'timestamp', 'date', 'Unnamed: 0'
]

# ═══════════════════════════════════════════════════════════════════════
# GPU Configuration
# ═══════════════════════════════════════════════════════════════════════

def configure_gpu():
    """Configure GPU with proper error handling."""
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU configured: {len(gpus)} device(s)")
            return True
        except RuntimeError as e:
            print(f"⚠️  GPU configuration failed: {e}")
            return False
    else:
        print("⚠️  No GPU detected, using CPU")
        return False

# ═══════════════════════════════════════════════════════════════════════
# Custom Precision-Focused Metrics
# ═══════════════════════════════════════════════════════════════════════

def precision_weighted_score(y_true, y_pred, precision_weight=2.0):
    """
    Custom metric that emphasizes precision over recall.
    
    Formula: (1 + precision_weight²) * precision * recall / (precision_weight² * precision + recall)
    This is similar to F-beta but with explicit precision weighting.
    """
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    if precision + recall == 0:
        return 0.0
    
    # Custom precision-weighted F-score
    score = (1 + precision_weight**2) * precision * recall / (precision_weight**2 * precision + recall)
    return score

def conservative_precision_score(y_true, y_pred):
    """
    Conservative precision metric that penalizes low precision heavily.
    Combines precision with penalty for low confidence predictions.
    """
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    # Penalty for low precision (precision < 0.6 gets heavily penalized)
    precision_penalty = max(0, (precision - 0.6) / 0.4) if precision > 0.6 else 0
    
    # Bonus for high precision (precision > 0.7 gets bonus)
    precision_bonus = max(0, (precision - 0.7) / 0.3) if precision > 0.7 else 0
    
    # Base score with precision emphasis
    base_score = precision * 0.7 + recall * 0.3  # 70% precision, 30% recall
    
    # Apply bonuses and penalties
    final_score = base_score + precision_bonus * 0.2 - (1 - precision_penalty) * 0.3
    
    return max(0, min(1, final_score))

# ═══════════════════════════════════════════════════════════════════════
# Data Loading and Preprocessing
# ═══════════════════════════════════════════════════════════════════════

def load_and_preprocess_data():
    """Load and preprocess data with comprehensive validation."""
    
    print("🚀 Loading Bitcoin data for PRECISION-optimized GRU...")
    print("=" * 60)
    
    if not CSV_PATH.exists():
        raise FileNotFoundError(f"❌ Data file not found: {CSV_PATH}")
    
    df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
    print(f"✅ Loaded data: {df.shape}")
    
    # Filter to stable period
    df = df[df.index >= "2020-01-01"]
    print(f"📅 Using data from 2020+: {len(df)} rows")
    
    if 'target' not in df.columns:
        raise ValueError("❌ Target column not found!")
    
    target_dist = df['target'].value_counts().to_dict()
    print(f"🎯 Target distribution: {target_dist}")
    
    # Drop columns
    cols_to_drop = [c for c in DROP_COLS if c in df.columns]
    df = df.drop(columns=cols_to_drop)
    df = df[df["target"].notna()].dropna()
    print(f"🧹 After cleaning: {df.shape}")
    
    # Validate no data leakage
    features = df.drop(columns="target")
    forbidden_cols = ['open', 'high', 'low', 'close']
    found_forbidden = [col for col in forbidden_cols if col in features.columns]
    
    if found_forbidden:
        raise ValueError(f"❌ DATA LEAKAGE: {found_forbidden} in features!")
    
    print(f"✅ Features ({len(features.columns)}): {list(features.columns)}")
    print(f"✅ PRECISION OPTIMIZATION: No price data leakage")
    
    target = df["target"].astype(int).values
    
    if len(df) < 1000:
        raise ValueError(f"❌ Insufficient data: {len(df)} rows")
    
    return features, target, df.index

# ═══════════════════════════════════════════════════════════════════════
# Sequence Creation
# ═══════════════════════════════════════════════════════════════════════

def create_sequences(data, labels, window_size):
    """Create sequences with validation."""
    if window_size >= len(data):
        raise ValueError(f"Window {window_size} >= data length {len(data)}")
    
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(labels[i])
    
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int8)

# ═══════════════════════════════════════════════════════════════════════
# Precision-Optimized Objective Function
# ═══════════════════════════════════════════════════════════════════════

def precision_objective(trial, data_info):
    """
    Objective function optimized for PRECISION.
    Uses multiple precision-focused metrics.
    """
    
    X_train_scaled, X_val_scaled, y_train, y_val = data_info
    
    try:
        # Sample hyperparameters (slightly more conservative for precision)
        window_size = trial.suggest_int("window", 12, 48, step=6)
        n_units = trial.suggest_int("units", 32, 128, step=32)
        n_layers = trial.suggest_int("layers", 1, 2)
        dropout_rate = trial.suggest_float("dropout", 0.2, 0.5)  # Higher dropout for precision
        learning_rate = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
        l2_reg = trial.suggest_float("l2", 1e-5, 1e-2, log=True)  # More regularization
        batch_size = trial.suggest_categorical("batch", [32, 64])
        
        # PRECISION OPTIMIZATION: Threshold tuning
        decision_threshold = trial.suggest_float("threshold", 0.4, 0.7)  # Conservative thresholds
        
        # Create sequences
        X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, window_size)
        X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val, window_size)
        
        if len(X_train_seq) < 100 or len(X_val_seq) < 50:
            return float('inf')
        
        if len(np.unique(y_train_seq)) < 2 or len(np.unique(y_val_seq)) < 2:
            return float('inf')
        
        # Build model (more conservative architecture for precision)
        tf.keras.backend.clear_session()
        model = keras.Sequential()
        
        for i in range(n_layers):
            return_sequences = (i < n_layers - 1)
            model.add(keras.layers.GRU(
                n_units,
                return_sequences=return_sequences,
                dropout=dropout_rate,
                recurrent_dropout=dropout_rate * 0.6,  # Higher recurrent dropout
                kernel_regularizer=keras.regularizers.l2(l2_reg),
                recurrent_regularizer=keras.regularizers.l2(l2_reg)
            ))
        
        # Add extra regularization for precision
        model.add(keras.layers.Dropout(dropout_rate))
        model.add(keras.layers.Dense(1, activation="sigmoid"))
        
        # Compile model
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss="binary_crossentropy",
            metrics=['accuracy']
        )
        
        # Train model with patience for precision
        history = model.fit(
            X_train_seq, y_train_seq,
            epochs=80,
            batch_size=batch_size,
            validation_data=(X_val_seq, y_val_seq),
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=15,  # More patience for precision optimization
                    restore_best_weights=True,
                    verbose=0
                )
            ],
            verbose=0
        )
        
        # Evaluate with custom threshold
        y_pred_prob = model.predict(X_val_seq, verbose=0).ravel()
        y_pred = (y_pred_prob >= decision_threshold).astype(int)
        
        # Calculate precision-focused metrics
        precision = precision_score(y_val_seq, y_pred, zero_division=0)
        recall = recall_score(y_val_seq, y_pred, zero_division=0)
        f1 = f1_score(y_val_seq, y_pred, zero_division=0)
        f_half = fbeta_score(y_val_seq, y_pred, beta=BETA, zero_division=0)  # F0.5 (precision-focused)
        accuracy = accuracy_score(y_val_seq, y_pred)
        
        # Custom precision metrics
        precision_weighted = precision_weighted_score(y_val_seq, y_pred, PRECISION_WEIGHT)
        conservative_precision = conservative_precision_score(y_val_seq, y_pred)
        
        # Store all metrics
        trial.set_user_attr("precision", precision)
        trial.set_user_attr("recall", recall)
        trial.set_user_attr("f1_score", f1)
        trial.set_user_attr("f_half_score", f_half)  # F0.5 score
        trial.set_user_attr("accuracy", accuracy)
        trial.set_user_attr("precision_weighted", precision_weighted)
        trial.set_user_attr("conservative_precision", conservative_precision)
        trial.set_user_attr("decision_threshold", decision_threshold)
        trial.set_user_attr("n_positive_predictions", int(np.sum(y_pred)))
        
        # PRECISION OPTIMIZATION: Multiple scoring strategies
        
        # Strategy 1: Pure F0.5 (emphasizes precision)
        score_f_half = f_half
        
        # Strategy 2: Precision-weighted custom score
        score_custom = precision_weighted
        
        # Strategy 3: Conservative precision score
        score_conservative = conservative_precision
        
        # Strategy 4: Precision with minimum recall constraint
        if recall >= 0.2:  # Minimum 20% recall required
            score_constrained = precision
        else:
            score_constrained = precision * (recall / 0.2)  # Penalty for low recall
        
        # Combine strategies (weighted average)
        final_score = (
            0.4 * score_f_half +           # 40% F0.5 score
            0.3 * score_custom +           # 30% custom precision score  
            0.2 * score_conservative +     # 20% conservative score
            0.1 * score_constrained        # 10% constrained precision
        )
        
        trial.set_user_attr("final_combined_score", final_score)
        
        # Return negative score for minimization
        return 1.0 - final_score
        
    except Exception as e:
        print(f"❌ Trial {trial.number} failed: {e}")
        return float('inf')

# ═══════════════════════════════════════════════════════════════════════
# Enhanced Progress Callback
# ═══════════════════════════════════════════════════════════════════════

def precision_progress_callback(study, trial):
    """Progress callback focused on precision metrics."""
    
    if trial.state == optuna.trial.TrialState.COMPLETE:
        attrs = trial.user_attrs
        precision = attrs.get('precision', 0)
        recall = attrs.get('recall', 0)
        f_half = attrs.get('f_half_score', 0)
        threshold = attrs.get('decision_threshold', 0.5)
        n_pos_pred = attrs.get('n_positive_predictions', 0)
        
        best_score = 1 - study.best_value if study.best_value != float('inf') else 0
        
        print(f"Trial {trial.number:2d}: P={precision:.3f} R={recall:.3f} F0.5={f_half:.3f} "
              f"Thr={threshold:.2f} Pos={n_pos_pred} | Best={best_score:.3f}")
    
    elif trial.state == optuna.trial.TrialState.PRUNED:
        print(f"Trial {trial.number:2d}: PRUNED")

# ═══════════════════════════════════════════════════════════════════════
# Main Precision Optimization Pipeline
# ═══════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    print("🚀 PRECISION-OPTIMIZED GRU HYPERPARAMETER SEARCH")
    print("🎯 Maximizing PRECISION for Bitcoin Direction Prediction")
    print("=" * 70)
    print(f"📊 Using F{BETA} score (β={BETA}) - Emphasizes PRECISION")
    print(f"🎯 Precision gets {PRECISION_WEIGHT}x weight in scoring")
    
    try:
        # Configure GPU
        gpu_available = configure_gpu()
        
        # Load and preprocess data
        features, target, dates = load_and_preprocess_data()
        
        # Proper scaling
        split_idx = int(len(features) * (1 - VAL_FRAC))
        
        scaler = StandardScaler()
        scaler.fit(features.iloc[:split_idx])
        
        X_train_scaled = scaler.transform(features.iloc[:split_idx])
        X_val_scaled = scaler.transform(features.iloc[split_idx:])
        y_train = target[:split_idx]
        y_val = target[split_idx:]
        
        print(f"📊 Train: {len(X_train_scaled)} samples, Val: {len(X_val_scaled)} samples")
        
        # Save scaler
        joblib.dump(scaler, "gru_precision_scaler.pkl")
        
        # Prepare data
        data_info = (X_train_scaled, X_val_scaled, y_train, y_val)
        
        # Create study
        study = optuna.create_study(
            direction="minimize",
            sampler=optuna.samplers.TPESampler(seed=SEED),
            pruner=optuna.pruners.MedianPruner(
                n_startup_trials=5,
                n_warmup_steps=3
            ),
            study_name="gru_precision_optimized"
        )
        
        # Run optimization
        print(f"\n🔍 Starting PRECISION optimization...")
        print(f"   Focus: Maximize precision with 2x weight")
        print(f"   Trials: {N_TRIALS}, Timeout: {TIMEOUT//60}min")
        
        start_time = time.time()
        
        study.optimize(
            lambda trial: precision_objective(trial, data_info),
            n_trials=N_TRIALS,
            timeout=TIMEOUT,
            show_progress_bar=True,
            callbacks=[precision_progress_callback]
        )
        
        total_time = time.time() - start_time
        
        # Results analysis
        print(f"\n🏆 PRECISION OPTIMIZATION COMPLETED")
        print("=" * 50)
        
        completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
        
        if completed_trials:
            best_combined_score = 1 - study.best_value
            best_trial = study.best_trial
            
            print(f"⭐ Best combined score: {best_combined_score:.4f}")
            print(f"📊 Completed trials: {len(completed_trials)}")
            print(f"⏱️  Total time: {total_time/60:.1f} minutes")
            
            # Best trial precision metrics
            if hasattr(best_trial, 'user_attrs'):
                attrs = best_trial.user_attrs
                print(f"\n🎯 BEST TRIAL PRECISION METRICS:")
                print(f"   Precision: {attrs.get('precision', 'N/A'):.4f} ⭐")
                print(f"   Recall: {attrs.get('recall', 'N/A'):.4f}")
                print(f"   F0.5-score: {attrs.get('f_half_score', 'N/A'):.4f} (precision-focused)")
                print(f"   F1-score: {attrs.get('f1_score', 'N/A'):.4f}")
                print(f"   Decision threshold: {attrs.get('decision_threshold', 'N/A'):.3f}")
                print(f"   Positive predictions: {attrs.get('n_positive_predictions', 'N/A')}")
            
            print(f"\n🎯 BEST PARAMETERS:")
            for param, value in study.best_params.items():
                print(f"   {param:15s}: {value}")
            
            # Save results
            enhanced_params = {
                **study.best_params,
                "optimization_focus": "precision",
                "beta_score": BETA,
                "precision_weight": PRECISION_WEIGHT,
                "best_metrics": {
                    k: float(v) if isinstance(v, (int, float, np.number)) else v
                    for k, v in best_trial.user_attrs.items()
                    if k in ['precision', 'recall', 'f_half_score', 'f1_score', 'decision_threshold']
                }
            }
            
            with open("gru_precision_optimized_params.json", "w") as f:
                json.dump(enhanced_params, f, indent=2)
            
            print(f"\n✅ Precision-optimized parameters saved!")
            print(f"🎯 Model optimized for HIGH PRECISION trading signals")
            
        else:
            print("❌ No trials completed successfully!")
    
    except Exception as e:
        print(f"❌ Optimization failed: {e}")
        import traceback
        traceback.print_exc()

[I 2025-06-07 12:55:45,523] A new study created in memory with name: gru_precision_optimized


🚀 PRECISION-OPTIMIZED GRU HYPERPARAMETER SEARCH
🎯 Maximizing PRECISION for Bitcoin Direction Prediction
📊 Using F0.5 score (β=0.5) - Emphasizes PRECISION
🎯 Precision gets 2.0x weight in scoring
⚠️  No GPU detected, using CPU
🚀 Loading Bitcoin data for PRECISION-optimized GRU...
✅ Loaded data: (20718, 66)
📅 Using data from 2020+: 11476 rows
🎯 Target distribution: {1: 5858, 0: 5618}
🧹 After cleaning: (11476, 20)
✅ Features (19): ['volume', 'RSI', 'MACD_histogram', 'OBV', 'stoch_%K', 'atr_14', 'atr_ratio', 'price_vs_vwap', 'volume_ratio', 'buying_pressure', 'adx', 'fear_greed_score', 'roc_4h', 'roc_24h', 'bb_position', 'macd_rising', 'obv_rising_24h', 'momentum_alignment', 'trend_alignment']
✅ PRECISION OPTIMIZATION: No price data leakage
📊 Train: 9180 samples, Val: 2296 samples

🔍 Starting PRECISION optimization...
   Focus: Maximize precision with 2x weight
   Trials: 30, Timeout: 100min


Best trial: 0. Best value: 0.981382:   3%|▎         | 1/30 [06:53<3:19:57, 413.71s/it, 413.71/6000 seconds]

[I 2025-06-07 13:02:39,228] Trial 0 finished with value: 0.9813824931254083 and parameters: {'window': 24, 'units': 128, 'layers': 2, 'dropout': 0.379597545259111, 'lr': 0.00018410729205738696, 'l2': 2.9375384576328295e-05, 'batch': 64, 'threshold': 0.5803345035229626}. Best is trial 0 with value: 0.9813824931254083.
Trial  0: P=0.409 R=0.008 F0.5=0.036 Thr=0.58 Pos=22 | Best=0.019


Best trial: 0. Best value: 0.981382:   7%|▋         | 2/30 [10:45<2:23:00, 306.45s/it, 645.08/6000 seconds]

...

[I 2025-06-07 14:23:39,902] Trial 27 finished with value: 0.3919346379287192 and parameters: {'window': 36, 'units': 64, 'layers': 1, 'dropout': 0.34882690126287674, 'lr': 0.0007750288765009825, 'l2': 0.0007786020101904646, 'batch': 32, 'threshold': 0.42606199161643293}. Best is trial 22 with value: 0.3918097792402703.
Trial 27: P=0.520 R=1.000 F0.5=0.575 Thr=0.43 Pos=2260 | Best=0.608


Best trial: 22. Best value: 0.39181:  97%|█████████▋| 29/30 [1:28:31<01:23, 83.40s/it, 5311.20/6000 seconds] 

[I 2025-06-07 14:24:16,725] Trial 28 finished with value: 0.39240567982827246 and parameters: {'window': 18, 'units': 32, 'layers': 1, 'dropout': 0.3454456702065626, 'lr': 0.003010191223030362, 'l2': 0.0002696124550195139, 'batch': 32, 'threshold': 0.4640763508312489}. Best is trial 22 with value: 0.3918097792402703.
Trial 28: P=0.519 R=1.000 F0.5=0.575 Thr=0.46 Pos=2278 | Best=0.608


Best trial: 22. Best value: 0.39181: 100%|██████████| 30/30 [1:29:41<00:00, 179.40s/it, 5381.86/6000 seconds]

[I 2025-06-07 14:25:27,379] Trial 29 finished with value: 1.0 and parameters: {'window': 30, 'units': 64, 'layers': 1, 'dropout': 0.37182564519122707, 'lr': 0.001233010707619071, 'l2': 5.347995878462169e-05, 'batch': 32, 'threshold': 0.5989922993489152}. Best is trial 22 with value: 0.3918097792402703.
Trial 29: P=0.000 R=0.000 F0.5=0.000 Thr=0.60 Pos=0 | Best=0.608

🏆 PRECISION OPTIMIZATION COMPLETED
⭐ Best combined score: 0.6082
📊 Completed trials: 30
⏱️  Total time: 89.7 minutes

🎯 BEST TRIAL PRECISION METRICS:
   Precision: 0.5215 ⭐
   Recall: 0.9957
   F0.5-score: 0.5764 (precision-focused)
   F1-score: 0.6845
   Decision threshold: 0.445
   Positive predictions: 2236

🎯 BEST PARAMETERS:
   window         : 42
   units          : 96
   layers         : 1
   dropout        : 0.29304784026893504
   lr             : 0.0009539147035893642
   l2             : 0.0006182071520515096
   batch          : 32
   threshold      : 0.4449592913770786

✅ Precision-optimized parameters saved!
🎯 M

In [2]:
"""
Enhanced Precision-Optimized GRU for Bitcoin Direction Prediction
=================================================================
FIXED VERSION with improved parameter ranges and more trials.
Focus: Maximize PRECISION with F0.5 score (2x weight to precision).
"""

import os
import json
import optuna
import warnings
import joblib
import time
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score, accuracy_score
import tensorflow as tf
from tensorflow import keras

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.get_logger().setLevel('ERROR')

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ═══════════════════════════════════════════════════════════════════════
# ENHANCED Configuration - PRECISION FOCUSED
# ═══════════════════════════════════════════════════════════════════════

CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
VAL_FRAC = 0.2

# PRECISION OPTIMIZATION: Use F0.5 score (emphasizes precision)
BETA = 0.5  # β=0.5 gives 2x weight to PRECISION
PRECISION_WEIGHT = 2.0

MODEL_NAME = "gru_precision_enhanced.h5"
N_TRIALS = 50  # INCREASED from 30
TIMEOUT = 120 * 60  # 2 hours instead of 100 minutes

# Complete drop list
DROP_COLS = [
    'open', 'high', 'low', 'close',
    'typical_price', 'vwap_24h', 'close_4h',
    'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'bollinger_upper', 'bollinger_lower', 'bollinger_width',
    'resistance_level', 'support_level',
    'high_low', 'high_close', 'low_close', 'true_range',
    'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'CCI', 'stoch_%D', 'parkinson_vol',
    'ema_cross_down', 'macd_cross_down', 'ema_cross_up', 'macd_cross_up',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'trending_market',
    'oversold_reversal', 'overbought_reversal',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'timestamp', 'date', 'Unnamed: 0'
]

# ═══════════════════════════════════════════════════════════════════════
# GPU Configuration
# ═══════════════════════════════════════════════════════════════════════

def configure_gpu():
    """Configure GPU with proper error handling."""
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU configured: {len(gpus)} device(s)")
            return True
        except RuntimeError as e:
            print(f"⚠️  GPU configuration failed: {e}")
            return False
    else:
        print("⚠️  No GPU detected, using CPU")
        return False

# ═══════════════════════════════════════════════════════════════════════
# Custom Precision-Focused Metrics
# ═══════════════════════════════════════════════════════════════════════

def precision_weighted_score(y_true, y_pred, precision_weight=2.0):
    """Custom metric that emphasizes precision over recall."""
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    if precision + recall == 0:
        return 0.0
    
    # Custom precision-weighted F-score
    score = (1 + precision_weight**2) * precision * recall / (precision_weight**2 * precision + recall)
    return score

def conservative_precision_score(y_true, y_pred):
    """Conservative precision metric with bonuses for high precision."""
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    # Bonus for high precision
    precision_bonus = max(0, (precision - 0.6) / 0.4) if precision > 0.6 else 0
    
    # Base score with precision emphasis
    base_score = precision * 0.7 + recall * 0.3
    
    # Apply bonus
    final_score = base_score + precision_bonus * 0.2
    
    return max(0, min(1, final_score))

# ═══════════════════════════════════════════════════════════════════════
# Data Loading and Preprocessing
# ═══════════════════════════════════════════════════════════════════════

def load_and_preprocess_data():
    """Load and preprocess data with comprehensive validation."""
    
    print("🚀 Loading Bitcoin data for ENHANCED PRECISION GRU...")
    print("=" * 65)
    
    if not CSV_PATH.exists():
        raise FileNotFoundError(f"❌ Data file not found: {CSV_PATH}")
    
    df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
    print(f"✅ Loaded data: {df.shape}")
    
    # Filter to stable period
    df = df[df.index >= "2020-01-01"]
    print(f"📅 Using data from 2020+: {len(df)} rows")
    
    if 'target' not in df.columns:
        raise ValueError("❌ Target column not found!")
    
    target_dist = df['target'].value_counts().to_dict()
    print(f"🎯 Target distribution: {target_dist}")
    
    # Drop columns
    cols_to_drop = [c for c in DROP_COLS if c in df.columns]
    df = df.drop(columns=cols_to_drop)
    df = df[df["target"].notna()].dropna()
    print(f"🧹 After cleaning: {df.shape}")
    
    # Validate no data leakage
    features = df.drop(columns="target")
    forbidden_cols = ['open', 'high', 'low', 'close']
    found_forbidden = [col for col in forbidden_cols if col in features.columns]
    
    if found_forbidden:
        raise ValueError(f"❌ DATA LEAKAGE: {found_forbidden} in features!")
    
    print(f"✅ Features ({len(features.columns)}): {list(features.columns)}")
    print(f"✅ ENHANCED PRECISION: No price data leakage")
    
    target = df["target"].astype(int).values
    
    if len(df) < 1000:
        raise ValueError(f"❌ Insufficient data: {len(df)} rows")
    
    return features, target, df.index

# ═══════════════════════════════════════════════════════════════════════
# Sequence Creation
# ═══════════════════════════════════════════════════════════════════════

def create_sequences(data, labels, window_size):
    """Create sequences with validation."""
    if window_size >= len(data):
        raise ValueError(f"Window {window_size} >= data length {len(data)}")
    
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(labels[i])
    
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int8)

# ═══════════════════════════════════════════════════════════════════════
# Enhanced Precision-Optimized Objective Function
# ═══════════════════════════════════════════════════════════════════════

def enhanced_precision_objective(trial, data_info):
    """ENHANCED objective function optimized for PRECISION."""
    
    X_train_scaled, X_val_scaled, y_train, y_val = data_info
    
    try:
        # ENHANCED hyperparameter ranges based on previous results
        window_size = trial.suggest_categorical("window", [18, 24, 30, 36, 42])  # Focus on successful windows
        n_units = trial.suggest_categorical("units", [64, 96, 128])  # Larger networks only
        n_layers = trial.suggest_int("layers", 1, 2)  # Simple architectures
        dropout_rate = trial.suggest_float("dropout", 0.15, 0.35)  # Tighter range
        learning_rate = trial.suggest_float("lr", 0.0005, 0.003, log=True)  # Focus on working LR range
        l2_reg = trial.suggest_float("l2", 1e-5, 1e-3, log=True)  # Better regularization
        batch_size = trial.suggest_categorical("batch", [32, 64])
        
        # ENHANCED threshold tuning - avoid extremes
        decision_threshold = trial.suggest_float("threshold", 0.35, 0.55)  # Avoid extreme thresholds
        
        # Create sequences
        X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, window_size)
        X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val, window_size)
        
        if len(X_train_seq) < 100 or len(X_val_seq) < 50:
            return float('inf')
        
        if len(np.unique(y_train_seq)) < 2 or len(np.unique(y_val_seq)) < 2:
            return float('inf')
        
        # Build enhanced model
        tf.keras.backend.clear_session()
        model = keras.Sequential()
        
        for i in range(n_layers):
            return_sequences = (i < n_layers - 1)
            model.add(keras.layers.GRU(
                n_units,
                return_sequences=return_sequences,
                dropout=dropout_rate,
                recurrent_dropout=dropout_rate * 0.6,
                kernel_regularizer=keras.regularizers.l2(l2_reg),
                recurrent_regularizer=keras.regularizers.l2(l2_reg)
            ))
        
        # Enhanced regularization
        model.add(keras.layers.Dropout(dropout_rate))
        model.add(keras.layers.Dense(1, activation="sigmoid"))
        
        # Compile model
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss="binary_crossentropy",
            metrics=['accuracy']
        )
        
        # Train with enhanced patience
        history = model.fit(
            X_train_seq, y_train_seq,
            epochs=60,  # Reduced epochs for efficiency
            batch_size=batch_size,
            validation_data=(X_val_seq, y_val_seq),
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=12,  # Balanced patience
                    restore_best_weights=True,
                    verbose=0
                )
            ],
            verbose=0
        )
        
        # Evaluate with custom threshold
        y_pred_prob = model.predict(X_val_seq, verbose=0).ravel()
        y_pred = (y_pred_prob >= decision_threshold).astype(int)
        
        # Calculate all metrics
        precision = precision_score(y_val_seq, y_pred, zero_division=0)
        recall = recall_score(y_val_seq, y_pred, zero_division=0)
        f1 = f1_score(y_val_seq, y_pred, zero_division=0)
        f_half = fbeta_score(y_val_seq, y_pred, beta=BETA, zero_division=0)
        accuracy = accuracy_score(y_val_seq, y_pred)
        
        # Custom precision metrics
        precision_weighted = precision_weighted_score(y_val_seq, y_pred, PRECISION_WEIGHT)
        conservative_precision = conservative_precision_score(y_val_seq, y_pred)
        
        # Store metrics
        trial.set_user_attr("precision", precision)
        trial.set_user_attr("recall", recall)
        trial.set_user_attr("f1_score", f1)
        trial.set_user_attr("f_half_score", f_half)
        trial.set_user_attr("accuracy", accuracy)
        trial.set_user_attr("precision_weighted", precision_weighted)
        trial.set_user_attr("conservative_precision", conservative_precision)
        trial.set_user_attr("decision_threshold", decision_threshold)
        trial.set_user_attr("n_positive_predictions", int(np.sum(y_pred)))
        
        # ENHANCED scoring with focus on precision
        score_f_half = f_half
        score_custom = precision_weighted
        score_conservative = conservative_precision
        
        # Precision with minimum recall constraint (enhanced)
        if recall >= 0.15:  # Lower minimum recall threshold
            score_constrained = precision
        else:
            score_constrained = precision * (recall / 0.15)
        
        # Enhanced combination weights
        final_score = (
            0.45 * score_f_half +         # 45% F0.5 score (increased)
            0.30 * score_custom +         # 30% custom precision score  
            0.15 * score_conservative +   # 15% conservative score
            0.10 * score_constrained      # 10% constrained precision
        )
        
        trial.set_user_attr("final_combined_score", final_score)
        
        # Cleanup memory
        del model
        tf.keras.backend.clear_session()
        gc.collect()
        
        return 1.0 - final_score
        
    except Exception as e:
        print(f"❌ Trial {trial.number} failed: {e}")
        tf.keras.backend.clear_session()
        gc.collect()
        return float('inf')

# ═══════════════════════════════════════════════════════════════════════
# Enhanced Progress Callback
# ═══════════════════════════════════════════════════════════════════════

def enhanced_precision_callback(study, trial):
    """Enhanced progress callback with detailed metrics."""
    
    if trial.state == optuna.trial.TrialState.COMPLETE:
        attrs = trial.user_attrs
        precision = attrs.get('precision', 0)
        recall = attrs.get('recall', 0)
        f1 = attrs.get('f1_score', 0)
        f_half = attrs.get('f_half_score', 0)
        threshold = attrs.get('decision_threshold', 0.5)
        combined_score = attrs.get('final_combined_score', 0)
        n_pos = attrs.get('n_positive_predictions', 0)
        
        best_score = 1 - study.best_value if study.best_value != float('inf') else 0
        
        print(f"✅ Trial {trial.number:2d} | P={precision:.3f} R={recall:.3f} F1={f1:.3f} "
              f"F0.5={f_half:.3f} | Thr={threshold:.2f} Pos={n_pos} | Combined={combined_score:.3f} | Best={best_score:.3f}")
        
        # Highlight high precision trials
        if precision >= 0.55:
            print(f"   🎯 HIGH PRECISION TRIAL! Conservative={attrs.get('conservative_precision', 0):.3f}")
    
    elif trial.state == optuna.trial.TrialState.PRUNED:
        print(f"⏭️  Trial {trial.number:2d}: PRUNED")
    elif trial.state == optuna.trial.TrialState.FAIL:
        print(f"❌ Trial {trial.number:2d}: FAILED")

# ═══════════════════════════════════════════════════════════════════════
# Main Enhanced Precision Optimization Pipeline
# ═══════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    print("🚀 ENHANCED PRECISION-OPTIMIZED GRU SEARCH")
    print("🎯 Maximizing PRECISION with improved parameters")
    print("=" * 70)
    print(f"📊 Using F{BETA} score (β={BETA}) - 2x weight to PRECISION")
    print(f"🎯 Enhanced with {N_TRIALS} trials and focused ranges")
    
    try:
        # Configure GPU
        gpu_available = configure_gpu()
        
        # Load and preprocess data
        features, target, dates = load_and_preprocess_data()
        
        # Proper scaling
        split_idx = int(len(features) * (1 - VAL_FRAC))
        
        scaler = StandardScaler()
        scaler.fit(features.iloc[:split_idx])
        
        X_train_scaled = scaler.transform(features.iloc[:split_idx])
        X_val_scaled = scaler.transform(features.iloc[split_idx:])
        y_train = target[:split_idx]
        y_val = target[split_idx:]
        
        print(f"📊 Train: {len(X_train_scaled)} samples, Val: {len(X_val_scaled)} samples")
        
        # Save scaler
        joblib.dump(scaler, "gru_enhanced_precision_scaler.pkl")
        
        # Prepare data
        data_info = (X_train_scaled, X_val_scaled, y_train, y_val)
        
        # Create enhanced study
        study = optuna.create_study(
            direction="minimize",
            sampler=optuna.samplers.TPESampler(seed=SEED),
            pruner=optuna.pruners.MedianPruner(
                n_startup_trials=8,  # More startup trials
                n_warmup_steps=5
            ),
            study_name="gru_enhanced_precision"
        )
        
        # Run enhanced optimization
        print(f"\n🔍 Starting ENHANCED PRECISION optimization...")
        print(f"   Focus: Precision with focused parameter ranges")
        print(f"   Trials: {N_TRIALS}, Timeout: {TIMEOUT//60}min")
        
        start_time = time.time()
        
        study.optimize(
            lambda trial: enhanced_precision_objective(trial, data_info),
            n_trials=N_TRIALS,
            timeout=TIMEOUT,
            show_progress_bar=True,
            callbacks=[enhanced_precision_callback]
        )
        
        total_time = time.time() - start_time
        
        # Enhanced results analysis
        print(f"\n🏆 ENHANCED PRECISION OPTIMIZATION COMPLETED")
        print("=" * 55)
        
        completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
        
        if completed_trials:
            best_combined_score = 1 - study.best_value
            best_trial = study.best_trial
            
            print(f"⭐ Best combined score: {best_combined_score:.4f}")
            print(f"📊 Completed trials: {len(completed_trials)}")
            print(f"⏱️  Total time: {total_time/60:.1f} minutes")
            
            # Best trial metrics
            if hasattr(best_trial, 'user_attrs'):
                attrs = best_trial.user_attrs
                print(f"\n🎯 BEST ENHANCED PRECISION METRICS:")
                print(f"   Precision: {attrs.get('precision', 'N/A'):.4f} ⭐")
                print(f"   Recall: {attrs.get('recall', 'N/A'):.4f}")
                print(f"   F0.5-score: {attrs.get('f_half_score', 'N/A'):.4f}")
                print(f"   F1-score: {attrs.get('f1_score', 'N/A'):.4f}")
                print(f"   Decision threshold: {attrs.get('decision_threshold', 'N/A'):.3f}")
                print(f"   Positive predictions: {attrs.get('n_positive_predictions', 'N/A')}")
            
            print(f"\n🎯 ENHANCED BEST PARAMETERS:")
            for param, value in study.best_params.items():
                print(f"   {param:15s}: {value}")
            
            # Save enhanced results
            enhanced_params = {
                **study.best_params,
                "optimization_type": "enhanced_precision",
                "beta_score": BETA,
                "precision_weight": PRECISION_WEIGHT,
                "n_trials": N_TRIALS,
                "best_metrics": {
                    k: float(v) if isinstance(v, (int, float, np.number)) else v
                    for k, v in best_trial.user_attrs.items()
                    if k in ['precision', 'recall', 'f_half_score', 'f1_score', 'decision_threshold', 'final_combined_score']
                }
            }
            
            with open("gru_enhanced_precision_params.json", "w") as f:
                json.dump(enhanced_params, f, indent=2)
            
            print(f"\n✅ Enhanced precision parameters saved!")
            print(f"🎯 Target: >0.60 precision with balanced recall")
            
        else:
            print("❌ No trials completed successfully!")
    
    except Exception as e:
        print(f"❌ Enhanced optimization failed: {e}")
        import traceback
        traceback.print_exc()

c:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-13 10:23:10,397] A new study created in memory with name: gru_enhanced_precision


🚀 ENHANCED PRECISION-OPTIMIZED GRU SEARCH
🎯 Maximizing PRECISION with improved parameters
📊 Using F0.5 score (β=0.5) - 2x weight to PRECISION
🎯 Enhanced with 50 trials and focused ranges
⚠️  No GPU detected, using CPU
🚀 Loading Bitcoin data for ENHANCED PRECISION GRU...
✅ Loaded data: (20718, 66)
📅 Using data from 2020+: 11476 rows
🎯 Target distribution: {1: 5858, 0: 5618}
🧹 After cleaning: (11476, 20)
✅ Features (19): ['volume', 'RSI', 'MACD_histogram', 'OBV', 'stoch_%K', 'atr_14', 'atr_ratio', 'price_vs_vwap', 'volume_ratio', 'buying_pressure', 'adx', 'fear_greed_score', 'roc_4h', 'roc_24h', 'bb_position', 'macd_rising', 'obv_rising_24h', 'momentum_alignment', 'trend_alignment']
✅ ENHANCED PRECISION: No price data leakage
📊 Train: 9180 samples, Val: 2296 samples

🔍 Starting ENHANCED PRECISION optimization...
   Focus: Precision with focused parameter ranges
   Trials: 50, Timeout: 120min


Best trial: 0. Best value: 0.337167:   2%|▏         | 1/50 [04:58<4:03:35, 298.27s/it, 298.27/7200 seconds]

[I 2025-06-13 10:28:08,666] Trial 0 finished with value: 0.3371671041142523 and parameters: {'window': 24, 'units': 128, 'layers': 2, 'dropout': 0.2916145155592091, 'lr': 0.0005187855301194422, 'l2': 0.0008706020878304854, 'batch': 32, 'threshold': 0.3863649934414201}. Best is trial 0 with value: 0.3371671041142523.
✅ Trial  0 | P=0.519 R=1.000 F1=0.683 F0.5=0.574 | Thr=0.39 Pos=2272 | Combined=0.663 | Best=0.663


Best trial: 0. Best value: 0.337167:   4%|▍         | 2/50 [05:36<1:56:26, 145.56s/it, 336.93/7200 seconds]

...

[I 2025-06-13 12:18:48,844] Trial 35 finished with value: 0.33637481028500593 and parameters: {'window': 36, 'units': 96, 'layers': 2, 'dropout': 0.3497330817000635, 'lr': 0.00210390084930548, 'l2': 9.726370534741423e-05, 'batch': 32, 'threshold': 0.43060993423577765}. Best is trial 11 with value: 0.33637481028500593.
✅ Trial 35 | P=0.520 R=1.000 F1=0.684 F0.5=0.575 | Thr=0.43 Pos=2260 | Combined=0.664 | Best=0.664


Best trial: 11. Best value: 0.336375:  74%|███████▍  | 37/50 [1:57:20<43:31, 200.85s/it, 7040.09/7200 seconds]

[I 2025-06-13 12:20:30,491] Trial 36 finished with value: 0.33637481028500593 and parameters: {'window': 36, 'units': 64, 'layers': 1, 'dropout': 0.32410843910799725, 'lr': 0.001593853375642476, 'l2': 0.0003939823902581817, 'batch': 64, 'threshold': 0.394527650935701}. Best is trial 11 with value: 0.33637481028500593.
✅ Trial 36 | P=0.520 R=1.000 F1=0.684 F0.5=0.575 | Thr=0.39 Pos=2260 | Combined=0.664 | Best=0.664


Best trial: 11. Best value: 0.336375:  76%|███████▌  | 38/50 [2:00:50<38:09, 190.81s/it, 7250.66/7200 seconds]

[I 2025-06-13 12:24:01,057] Trial 37 finished with value: 0.33677193249468607 and parameters: {'window': 30, 'units': 96, 'layers': 2, 'dropout': 0.267142572545942, 'lr': 0.000808836272445403, 'l2': 0.0007783687973588303, 'batch': 32, 'threshold': 0.4199580965645373}. Best is trial 11 with value: 0.33637481028500593.
✅ Trial 37 | P=0.519 R=1.000 F1=0.684 F0.5=0.575 | Thr=0.42 Pos=2266 | Combined=0.663 | Best=0.664

🏆 ENHANCED PRECISION OPTIMIZATION COMPLETED
⭐ Best combined score: 0.6636
📊 Completed trials: 38
⏱️  Total time: 120.8 minutes

🎯 BEST ENHANCED PRECISION METRICS:
   Precision: 0.5199 ⭐
   Recall: 1.0000
   F0.5-score: 0.5751
   F1-score: 0.6841
   Decision threshold: 0.477
   Positive predictions: 2260

🎯 ENHANCED BEST PARAMETERS:
   window         : 36
   units          : 96
   layers         : 2
   dropout        : 0.323957750393217
   lr             : 0.0015190233563579104
   l2             : 0.0009583723340357708
   batch          : 32
   threshold      : 0.477167569944

In [3]:
"""
gru_multi_config_test.py
────────────────────────────────────────────────────────────
Tests multiple GRU configurations from optimization results.
Trains each config and provides detailed performance summary.
Identifies the best performing configuration for final training.
"""

import os
import json
import gc
import warnings
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score, f1_score, 
                             fbeta_score, accuracy_score, roc_auc_score,
                             confusion_matrix, classification_report)
import tensorflow as tf
from tensorflow import keras

# ══════════════════════════════════════════════════════════════════════
# Setup
# ══════════════════════════════════════════════════════════════════════
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Configure GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU configured: {len(gpus)} device(s)")
    except RuntimeError as e:
        print(f"⚠️  GPU configuration failed: {e}")
else:
    print("⚠️  No GPU detected, using CPU")

# ══════════════════════════════════════════════════════════════════════
# Configuration
# ══════════════════════════════════════════════════════════════════════
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

VAL_FRAC = 0.20
BETA = 0.5  # For F0.5 score (precision-focused)
EPOCHS = 60
EARLY_STOP = 15

# Complete drop columns list (same as optimization)
DROP_COLS = [
    'open', 'high', 'low', 'close',
    'typical_price', 'vwap_24h', 'close_4h',
    'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'bollinger_upper', 'bollinger_lower', 'bollinger_width',
    'resistance_level', 'support_level',
    'high_low', 'high_close', 'low_close', 'true_range',
    'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'CCI', 'stoch_%D', 'parkinson_vol',
    'ema_cross_down', 'macd_cross_down', 'ema_cross_up', 'macd_cross_up',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'trending_market',
    'oversold_reversal', 'overbought_reversal',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'timestamp', 'date', 'Unnamed: 0'
]

# Top 6 configurations from your optimization results (threshold fixed at 0.5)
TEST_CONFIGS = [
    {
        'name': 'Trial_4_Balanced',
        'window': 30, 'units': 128, 'layers': 1, 'dropout': 0.3473773873201034,
        'lr': 0.0019947718789028682, 'l2': 2.497073714505272e-05, 'batch': 64,
        'expected': {'precision': 0.547, 'recall': 0.424, 'f1': 0.478, 'f05': 0.517}
    },
    {
        'name': 'Trial_6_HighPrecision',
        'window': 42, 'units': 96, 'layers': 1, 'dropout': 0.17157828539866088,
        'lr': 0.0005289646680152981, 'l2': 0.000187422109855557, 'batch': 64,
        'expected': {'precision': 0.556, 'recall': 0.221, 'f1': 0.316, 'f05': 0.427}
    },
    {
        'name': 'Trial_8_MaxPrecision',
        'window': 24, 'units': 128, 'layers': 1, 'dropout': 0.2521494605155131,
        'lr': 0.0010562796147104163, 'l2': 2.781093697926551e-05, 'batch': 64,
        'expected': {'precision': 0.585, 'recall': 0.111, 'f1': 0.187, 'f05': 0.316}
    },
    {
        'name': 'Trial_16_Conservative',
        'window': 36, 'units': 96, 'layers': 2, 'dropout': 0.30094837842359834,
        'lr': 0.0012323935680785315, 'l2': 6.268172739785502e-05, 'batch': 32,
        'expected': {'precision': 0.577, 'recall': 0.213, 'f1': 0.311, 'f05': 0.430}
    },
    {
        'name': 'Trial_20_Moderate',
        'window': 36, 'units': 96, 'layers': 2, 'dropout': 0.26126299099534406,
        'lr': 0.0008574666921947649, 'l2': 4.0337715213162597e-05, 'batch': 32,
        'expected': {'precision': 0.530, 'recall': 0.723, 'f1': 0.612, 'f05': 0.560}
    },
    {
        'name': 'Trial_28_BestOverall',
        'window': 36, 'units': 96, 'layers': 2, 'dropout': 0.2944123891707492,
        'lr': 0.0013899204674669833, 'l2': 0.0002516233411563695, 'batch': 32,
        'expected': {'precision': 0.522, 'recall': 0.991, 'f1': 0.683, 'f05': 0.576}
    }
]

# Fixed threshold for all configurations
DECISION_THRESHOLD = 0.5

# ══════════════════════════════════════════════════════════════════════
# Helper Functions
# ══════════════════════════════════════════════════════════════════════
def create_sequences(data, labels, window_size):
    """Create sequences for time series data."""
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(labels[i])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int8)

def build_gru_model(config, n_features):
    """Build GRU model with given configuration."""
    model = keras.Sequential()
    
    for i in range(config['layers']):
        return_sequences = (i < config['layers'] - 1)
        model.add(keras.layers.GRU(
            config['units'],
            return_sequences=return_sequences,
            dropout=config['dropout'],
            recurrent_dropout=config['dropout'] * 0.6,
            kernel_regularizer=keras.regularizers.l2(config['l2']),
            recurrent_regularizer=keras.regularizers.l2(config['l2'])
        ))
    
    model.add(keras.layers.Dropout(config['dropout']))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config['lr']),
        loss="binary_crossentropy",
        metrics=['accuracy']
    )
    
    return model

def calculate_metrics(y_true, y_pred, y_prob, config_name, expected_metrics):
    """Calculate comprehensive metrics."""
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    f05 = fbeta_score(y_true, y_pred, beta=BETA, zero_division=0)
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)
    
    # Calculate differences from expected
    precision_diff = precision - expected_metrics['precision']
    recall_diff = recall - expected_metrics['recall']
    f1_diff = f1 - expected_metrics['f1']
    f05_diff = f05 - expected_metrics['f05']
    
    metrics = {
        'config_name': config_name,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'f05_score': f05,
        'accuracy': accuracy,
        'auc': auc,
        'n_positive_predictions': int(np.sum(y_pred)),
        'expected_precision': expected_metrics['precision'],
        'expected_recall': expected_metrics['recall'],
        'expected_f1': expected_metrics['f1'],
        'expected_f05': expected_metrics['f05'],
        'precision_diff': precision_diff,
        'recall_diff': recall_diff,
        'f1_diff': f1_diff,
        'f05_diff': f05_diff,
        'confusion_matrix': confusion_matrix(y_true, y_pred).tolist()
    }
    
    return metrics

# ══════════════════════════════════════════════════════════════════════
# Data Loading and Preprocessing
# ══════════════════════════════════════════════════════════════════════
print("🚀 GRU Multi-Configuration Performance Test")
print("=" * 55)
print("📊 Loading and preprocessing data...")

# Load data
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df[df.index >= "2020-01-01"]  # Same as optimization
print(f"   Loaded data: {df.shape}")

# Clean data
if 'target' not in df.columns:
    raise ValueError("❌ Target column not found!")

# Drop columns
cols_to_drop = [c for c in DROP_COLS if c in df.columns]
df = df.drop(columns=cols_to_drop)
df = df[df["target"].notna()].dropna()
print(f"   After cleaning: {df.shape}")

# Prepare features and target
features = df.drop(columns="target")
target = df["target"].astype(int).values
n_features = features.shape[1]

print(f"   Features: {n_features}")
print(f"   Target distribution: {np.bincount(target)}")

# Scale features
split_idx = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler()
scaler.fit(features.iloc[:split_idx])

X_train_scaled = scaler.transform(features.iloc[:split_idx])
X_val_scaled = scaler.transform(features.iloc[split_idx:])
y_train = target[:split_idx]
y_val = target[split_idx:]

print(f"   Train samples: {len(X_train_scaled):,}")
print(f"   Val samples: {len(X_val_scaled):,}")

# Save scaler for later use
joblib.dump(scaler, "gru_multi_config_scaler.pkl")

# ══════════════════════════════════════════════════════════════════════
# Test Each Configuration
# ══════════════════════════════════════════════════════════════════════
results = []

for i, config in enumerate(TEST_CONFIGS, 1):
    print(f"\n{'='*70}")
    print(f"🔬 Testing Configuration {i}/6: {config['name']}")
    print(f"{'='*70}")
    
    # Display configuration
    print(f"   Window: {config['window']}, Units: {config['units']}, Layers: {config['layers']}")
    print(f"   Dropout: {config['dropout']:.3f}, LR: {config['lr']:.6f}, L2: {config['l2']:.2e}")
    print(f"   Batch: {config['batch']}, Threshold: {DECISION_THRESHOLD} (fixed)")
    
    # Expected performance
    exp = config['expected']
    print(f"   Expected: P={exp['precision']:.3f}, R={exp['recall']:.3f}, F1={exp['f1']:.3f}, F0.5={exp['f05']:.3f}")
    print(f"   Note: Expected values were from optimized thresholds, actual results may differ")
    
    try:
        # Create sequences
        X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, config['window'])
        X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val, config['window'])
        
        print(f"   Train sequences: {len(X_train_seq):,}")
        print(f"   Val sequences: {len(X_val_seq):,}")
        
        # Build model
        tf.keras.backend.clear_session()
        gc.collect()
        
        model = build_gru_model(config, n_features)
        
        print("   🚀 Training model...")
        start_time = datetime.now()
        
        # Train model
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=EARLY_STOP,
                restore_best_weights=True,
                verbose=0
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=7,
                min_lr=1e-7,
                verbose=0
            )
        ]
        
        history = model.fit(
            X_train_seq, y_train_seq,
            epochs=EPOCHS,
            batch_size=config['batch'],
            validation_data=(X_val_seq, y_val_seq),
            callbacks=callbacks,
            verbose=0
        )
        
        training_time = datetime.now() - start_time
        epochs_trained = len(history.history['loss'])
        
        print(f"   ⏱️  Training completed in {training_time} ({epochs_trained} epochs)")
        
        # Evaluate model
        y_pred_prob = model.predict(X_val_seq, verbose=0).ravel()
        y_pred = (y_pred_prob >= DECISION_THRESHOLD).astype(int)
        
        # Calculate metrics
        metrics = calculate_metrics(y_val_seq, y_pred, y_pred_prob, config['name'], config['expected'])
        metrics['training_time_seconds'] = training_time.total_seconds()
        metrics['epochs_trained'] = epochs_trained
        metrics['config'] = {k: v for k, v in config.items() if k not in ['name', 'expected']}
        metrics['config']['threshold_used'] = DECISION_THRESHOLD
        
        # Display results
        print(f"\n   📊 Results:")
        print(f"   ├─ Precision: {metrics['precision']:.3f} (expected: {exp['precision']:.3f}) "
              f"{'✅' if metrics['precision_diff'] >= -0.02 else '❌'} [{metrics['precision_diff']:+.3f}]")
        print(f"   ├─ Recall:    {metrics['recall']:.3f} (expected: {exp['recall']:.3f}) "
              f"{'✅' if metrics['recall_diff'] >= -0.02 else '❌'} [{metrics['recall_diff']:+.3f}]")
        print(f"   ├─ F1 Score:  {metrics['f1_score']:.3f} (expected: {exp['f1']:.3f}) "
              f"{'✅' if metrics['f1_diff'] >= -0.02 else '❌'} [{metrics['f1_diff']:+.3f}]")
        print(f"   ├─ F0.5 Score: {metrics['f05_score']:.3f} (expected: {exp['f05']:.3f}) "
              f"{'✅' if metrics['f05_diff'] >= -0.02 else '❌'} [{metrics['f05_diff']:+.3f}]")
        print(f"   ├─ Accuracy:  {metrics['accuracy']:.3f}")
        print(f"   ├─ AUC:       {metrics['auc']:.3f}")
        print(f"   └─ Positive predictions: {metrics['n_positive_predictions']}")
        
        # Save model if it meets expectations
        model_filename = f"gru_{config['name'].lower()}.h5"
        model.save(model_filename)
        print(f"   💾 Model saved: {model_filename}")
        
        results.append(metrics)
        
    except Exception as e:
        print(f"   ❌ Training failed: {str(e)}")
        error_metrics = {
            'config_name': config['name'],
            'error': str(e),
            'status': 'failed'
        }
        results.append(error_metrics)
    
    finally:
        # Cleanup
        tf.keras.backend.clear_session()
        gc.collect()

# ══════════════════════════════════════════════════════════════════════
# Final Analysis and Summary
# ══════════════════════════════════════════════════════════════════════
print(f"\n{'='*80}")
print("🏆 MULTI-CONFIGURATION TEST RESULTS")
print(f"{'='*80}")

# Filter successful results
successful_results = [r for r in results if 'error' not in r]

if successful_results:
    print(f"\n📊 Performance Summary (Successful: {len(successful_results)}/{len(TEST_CONFIGS)}):")
    print(f"{'':25s} {'Precision':>9s} {'Recall':>7s} {'F1':>6s} {'F0.5':>6s} {'AUC':>6s} {'Status':>8s}")
    print("-" * 80)
    
    for result in successful_results:
        status = "✅ Good" if result['precision_diff'] >= -0.02 and result['f05_diff'] >= -0.02 else "⚠️  Poor"
        print(f"{result['config_name']:25s} {result['precision']:>9.3f} {result['recall']:>7.3f} "
              f"{result['f1_score']:>6.3f} {result['f05_score']:>6.3f} {result['auc']:>6.3f} {status:>8s}")
    
    # Find best performers
    best_precision = max(successful_results, key=lambda x: x['precision'])
    best_f05 = max(successful_results, key=lambda x: x['f05_score'])
    best_balanced = max(successful_results, key=lambda x: x['f1_score'])
    
    print(f"\n🏆 Top Performers:")
    print(f"   🎯 Best Precision: {best_precision['config_name']} ({best_precision['precision']:.3f})")
    print(f"   📊 Best F0.5:      {best_f05['config_name']} ({best_f05['f05_score']:.3f})")
    print(f"   ⚖️  Best Balanced:  {best_balanced['config_name']} ({best_balanced['f1_score']:.3f})")
    
    # Save comprehensive results
    timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    summary = {
        "timestamp": timestamp + "Z",
        "test_type": "multi_configuration_comparison",
        "configurations_tested": len(TEST_CONFIGS),
        "successful_configs": len(successful_results),
        "dataset_info": {
            "total_samples": len(df),
            "train_samples": len(X_train_scaled),
            "val_samples": len(X_val_scaled),
            "features": n_features
        },
        "best_performers": {
            "best_precision": best_precision['config_name'],
            "best_f05": best_f05['config_name'],
            "best_balanced": best_balanced['config_name']
        },
        "detailed_results": results
    }
    
    results_filename = f"gru_multi_config_results_{timestamp}.json"
    with open(results_filename, "w") as f:
        json.dump(summary, f, indent=2)
    
    print(f"\n📁 Files Generated:")
    print(f"   • {results_filename} - Complete test results")
    print(f"   • gru_multi_config_scaler.pkl - Feature scaler")
    print(f"   • gru_trial_*.h5 - Individual trained models")
    
    print(f"\n💡 Recommendation:")
    if best_f05['f05_score'] >= 0.5:
        print(f"   Use {best_f05['config_name']} for precision-focused trading")
    elif best_balanced['f1_score'] >= 0.6:
        print(f"   Use {best_balanced['config_name']} for balanced performance")
    else:
        print(f"   Consider additional hyperparameter tuning or different approach")

else:
    print("❌ No configurations completed successfully!")

print(f"\n🎉 Multi-configuration test complete!")

⚠️  No GPU detected, using CPU
🚀 GRU Multi-Configuration Performance Test
📊 Loading and preprocessing data...
   Loaded data: (11476, 66)
   After cleaning: (11476, 20)
   Features: 19
   Target distribution: [5618 5858]
   Train samples: 9,180
   Val samples: 2,296

🔬 Testing Configuration 1/6: Trial_4_Balanced
   Window: 30, Units: 128, Layers: 1
   Dropout: 0.347, LR: 0.001995, L2: 2.50e-05
   Batch: 64, Threshold: 0.5 (fixed)
   Expected: P=0.547, R=0.424, F1=0.478, F0.5=0.517
   Note: Expected values were from optimized thresholds, actual results may differ
   Train sequences: 9,150
   Val sequences: 2,266
   🚀 Training model...
   ⏱️  Training completed in 0:01:25.188557 (41 epochs)



   📊 Results:
   ├─ Precision: 0.555 (expected: 0.547) ✅ [+0.008]
   ├─ Recall:    0.366 (expected: 0.424) ❌ [-0.058]
   ├─ F1 Score:  0.441 (expected: 0.478) ❌ [-0.037]
   ├─ F0.5 Score: 0.503 (expected: 0.517) ✅ [-0.014]
   ├─ Accuracy:  0.518
   ├─ AUC:       0.535
   └─ Positive predictions: 777
   💾 Model saved: gru_trial_4_balanced.h5

🔬 Testing Configuration 2/6: Trial_6_HighPrecision
   Window: 42, Units: 96, Layers: 1
   Dropout: 0.172, LR: 0.000529, L2: 1.87e-04
   Batch: 64, Threshold: 0.5 (fixed)
   Expected: P=0.556, R=0.221, F1=0.316, F0.5=0.427
   Note: Expected values were from optimized thresholds, actual results may differ
   Train sequences: 9,138
   Val sequences: 2,254
   🚀 Training model...
   ⏱️  Training completed in 0:02:49.410080 (60 epochs)



   📊 Results:
   ├─ Precision: 0.536 (expected: 0.556) ❌ [-0.020]
   ├─ Recall:    0.484 (expected: 0.221) ✅ [+0.263]
   ├─ F1 Score:  0.509 (expected: 0.316) ✅ [+0.193]
   ├─ F0.5 Score: 0.525 (expected: 0.427) ✅ [+0.098]
   ├─ Accuracy:  0.514
   ├─ AUC:       0.520
   └─ Positive predictions: 1058
   💾 Model saved: gru_trial_6_highprecision.h5

🔬 Testing Configuration 3/6: Trial_8_MaxPrecision
   Window: 24, Units: 128, Layers: 1
   Dropout: 0.252, LR: 0.001056, L2: 2.78e-05
   Batch: 64, Threshold: 0.5 (fixed)
   Expected: P=0.585, R=0.111, F1=0.187, F0.5=0.316
   Note: Expected values were from optimized thresholds, actual results may differ
   Train sequences: 9,156
   Val sequences: 2,272
   🚀 Training model...
   ⏱️  Training completed in 0:00:23.845321 (19 epochs)



   📊 Results:
   ├─ Precision: 0.527 (expected: 0.585) ❌ [-0.058]
   ├─ Recall:    0.435 (expected: 0.111) ✅ [+0.324]
   ├─ F1 Score:  0.477 (expected: 0.187) ✅ [+0.290]
   ├─ F0.5 Score: 0.505 (expected: 0.316) ✅ [+0.189]
   ├─ Accuracy:  0.504
   ├─ AUC:       0.515
   └─ Positive predictions: 974
   💾 Model saved: gru_trial_8_maxprecision.h5

🔬 Testing Configuration 4/6: Trial_16_Conservative
   Window: 36, Units: 96, Layers: 2
   Dropout: 0.301, LR: 0.001232, L2: 6.27e-05
   Batch: 32, Threshold: 0.5 (fixed)
   Expected: P=0.577, R=0.213, F1=0.311, F0.5=0.430
   Note: Expected values were from optimized thresholds, actual results may differ
   Train sequences: 9,144
   Val sequences: 2,260
   🚀 Training model...
   ⏱️  Training completed in 0:03:35.994361 (50 epochs)



   📊 Results:
   ├─ Precision: 0.543 (expected: 0.577) ❌ [-0.034]
   ├─ Recall:    0.384 (expected: 0.213) ✅ [+0.171]
   ├─ F1 Score:  0.450 (expected: 0.311) ✅ [+0.139]
   ├─ F0.5 Score: 0.501 (expected: 0.430) ✅ [+0.071]
   ├─ Accuracy:  0.512
   ├─ AUC:       0.529
   └─ Positive predictions: 831
   💾 Model saved: gru_trial_16_conservative.h5

🔬 Testing Configuration 5/6: Trial_20_Moderate
   Window: 36, Units: 96, Layers: 2
   Dropout: 0.261, LR: 0.000857, L2: 4.03e-05
   Batch: 32, Threshold: 0.5 (fixed)
   Expected: P=0.530, R=0.723, F1=0.612, F0.5=0.560
   Note: Expected values were from optimized thresholds, actual results may differ
   Train sequences: 9,144
   Val sequences: 2,260
   🚀 Training model...
   ⏱️  Training completed in 0:03:40.015653 (48 epochs)



   📊 Results:
   ├─ Precision: 0.537 (expected: 0.530) ✅ [+0.007]
   ├─ Recall:    0.386 (expected: 0.723) ❌ [-0.337]
   ├─ F1 Score:  0.449 (expected: 0.612) ❌ [-0.163]
   ├─ F0.5 Score: 0.498 (expected: 0.560) ❌ [-0.062]
   ├─ Accuracy:  0.508
   ├─ AUC:       0.529
   └─ Positive predictions: 843
   💾 Model saved: gru_trial_20_moderate.h5

🔬 Testing Configuration 6/6: Trial_28_BestOverall
   Window: 36, Units: 96, Layers: 2
   Dropout: 0.294, LR: 0.001390, L2: 2.52e-04
   Batch: 32, Threshold: 0.5 (fixed)
   Expected: P=0.522, R=0.991, F1=0.683, F0.5=0.576
   Note: Expected values were from optimized thresholds, actual results may differ
   Train sequences: 9,144
   Val sequences: 2,260
   🚀 Training model...
   ⏱️  Training completed in 0:06:04.638538 (49 epochs)



   📊 Results:
   ├─ Precision: 0.551 (expected: 0.522) ✅ [+0.029]
   ├─ Recall:    0.437 (expected: 0.991) ❌ [-0.554]
   ├─ F1 Score:  0.488 (expected: 0.683) ❌ [-0.195]
   ├─ F0.5 Score: 0.524 (expected: 0.576) ❌ [-0.052]
   ├─ Accuracy:  0.522
   ├─ AUC:       0.526
   └─ Positive predictions: 933
   💾 Model saved: gru_trial_28_bestoverall.h5

🏆 MULTI-CONFIGURATION TEST RESULTS

📊 Performance Summary (Successful: 6/6):
                          Precision  Recall     F1   F0.5    AUC   Status
--------------------------------------------------------------------------------
Trial_4_Balanced              0.555   0.366  0.441  0.503  0.535   ✅ Good
Trial_6_HighPrecision         0.536   0.484  0.509  0.525  0.520 ⚠️  Poor
Trial_8_MaxPrecision          0.527   0.435  0.477  0.505  0.515 ⚠️  Poor
Trial_16_Conservative         0.543   0.384  0.450  0.501  0.529 ⚠️  Poor
Trial_20_Moderate             0.537   0.386  0.449  0.498  0.529 ⚠️  Poor
Trial_28_BestOverall          0.551   0.437  0.48

In [4]:
"""
gru_trial28_final_training.py
─────────────────────────────────────────────────────────
Final training of Trial 28 (BestOverall) GRU configuration.
Trains the model and generates predictions CSV in the exact format requested.

Trial 28 Config: 0.551 precision, 0.437 recall, 0.488 F1, 0.524 F0.5
"""

import os
import json
import gc
import warnings
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score, f1_score, 
                             fbeta_score, accuracy_score, roc_auc_score,
                             confusion_matrix, classification_report)
import tensorflow as tf
from tensorflow import keras

# ══════════════════════════════════════════════════════════════════════
# Setup
# ══════════════════════════════════════════════════════════════════════
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Configure GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU configured: {len(gpus)} device(s)")
    except RuntimeError as e:
        print(f"⚠️  GPU configuration failed: {e}")
else:
    print("⚠️  No GPU detected, using CPU")

# ══════════════════════════════════════════════════════════════════════
# Configuration
# ══════════════════════════════════════════════════════════════════════
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

VAL_FRAC = 0.20
DECISION_THRESHOLD = 0.5
EPOCHS = 80
EARLY_STOP = 15

# Output files
MODEL_OUT = "gru_trial28_final.h5"
SCALER_OUT = "gru_trial28_scaler.pkl"
PREDICTIONS_OUT = "gru_trial28_predictions.csv"
SUMMARY_JSON = "gru_trial28_training_summary.json"

# Complete drop columns list (same as optimization)
DROP_COLS = [
    'open', 'high', 'low', 'close',
    'typical_price', 'vwap_24h', 'close_4h',
    'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'bollinger_upper', 'bollinger_lower', 'bollinger_width',
    'resistance_level', 'support_level',
    'high_low', 'high_close', 'low_close', 'true_range',
    'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'CCI', 'stoch_%D', 'parkinson_vol',
    'ema_cross_down', 'macd_cross_down', 'ema_cross_up', 'macd_cross_up',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'trending_market',
    'oversold_reversal', 'overbought_reversal',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'timestamp', 'date', 'Unnamed: 0'
]

# Trial 28 Optimal Configuration
TRIAL28_CONFIG = {
    'name': 'Trial_28_BestOverall',
    'window': 36,
    'units': 96,
    'layers': 2,
    'dropout': 0.2944123891707492,
    'lr': 0.0013899204674669833,
    'l2': 0.0002516233411563695,
    'batch': 32,
    'expected_performance': {
        'precision': 0.551,
        'recall': 0.437,
        'f1': 0.488,
        'f05': 0.524
    }
}

# ══════════════════════════════════════════════════════════════════════
# Helper Functions
# ══════════════════════════════════════════════════════════════════════
def create_sequences(data, labels, window_size):
    """Create sequences for time series data."""
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(labels[i])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int8)

def build_gru_model(config, n_features):
    """Build GRU model with Trial 28 configuration."""
    model = keras.Sequential()
    
    for i in range(config['layers']):
        return_sequences = (i < config['layers'] - 1)
        model.add(keras.layers.GRU(
            config['units'],
            return_sequences=return_sequences,
            dropout=config['dropout'],
            recurrent_dropout=config['dropout'] * 0.6,
            kernel_regularizer=keras.regularizers.l2(config['l2']),
            recurrent_regularizer=keras.regularizers.l2(config['l2'])
        ))
    
    model.add(keras.layers.Dropout(config['dropout']))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config['lr']),
        loss="binary_crossentropy",
        metrics=['accuracy']
    )
    
    return model

# ══════════════════════════════════════════════════════════════════════
# Data Loading and Preprocessing
# ══════════════════════════════════════════════════════════════════════
print("🚀 GRU Trial 28 Final Training")
print("=" * 45)
print("🎯 Configuration: BestOverall (0.551 precision, 0.437 recall)")
print("📊 Loading and preprocessing data...")

# Load data
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df[df.index >= "2020-01-01"]  # Same as optimization
print(f"   Loaded data: {df.shape}")

# Clean data
if 'target' not in df.columns:
    raise ValueError("❌ Target column not found!")

# Drop columns
cols_to_drop = [c for c in DROP_COLS if c in df.columns]
df = df.drop(columns=cols_to_drop)
df = df[df["target"].notna()].dropna()
print(f"   After cleaning: {df.shape}")

# Prepare features and target
features = df.drop(columns="target")
target = df["target"].astype(int).values
n_features = features.shape[1]

print(f"   Features: {n_features}")
print(f"   Target distribution: {np.bincount(target)}")

# Scale features
split_idx = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler()
scaler.fit(features.iloc[:split_idx])

X_train_scaled = scaler.transform(features.iloc[:split_idx])
X_val_scaled = scaler.transform(features.iloc[split_idx:])
y_train = target[:split_idx]
y_val = target[split_idx:]

print(f"   Train samples: {len(X_train_scaled):,}")
print(f"   Val samples: {len(X_val_scaled):,}")

# Save scaler
joblib.dump(scaler, SCALER_OUT)
print(f"   Scaler saved: {SCALER_OUT}")

# ══════════════════════════════════════════════════════════════════════
# Create Sequences
# ══════════════════════════════════════════════════════════════════════
print(f"\n🔄 Creating sequences (window={TRIAL28_CONFIG['window']})...")

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, TRIAL28_CONFIG['window'])
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val, TRIAL28_CONFIG['window'])

print(f"   Train sequences: {len(X_train_seq):,}")
print(f"   Val sequences: {len(X_val_seq):,}")

# ══════════════════════════════════════════════════════════════════════
# Build and Train Model
# ══════════════════════════════════════════════════════════════════════
print(f"\n🏗️ Building GRU model...")
print(f"   Units: {TRIAL28_CONFIG['units']}, Layers: {TRIAL28_CONFIG['layers']}")
print(f"   Dropout: {TRIAL28_CONFIG['dropout']:.3f}, LR: {TRIAL28_CONFIG['lr']:.6f}")
print(f"   L2: {TRIAL28_CONFIG['l2']:.2e}, Batch: {TRIAL28_CONFIG['batch']}")

tf.keras.backend.clear_session()
gc.collect()

model = build_gru_model(TRIAL28_CONFIG, n_features)

# Display model summary
print(f"\n📋 Model architecture:")
model.summary(line_length=100)

# Prepare callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=EARLY_STOP,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    )
]

print(f"\n🚀 Training model...")
print(f"   Epochs: {EPOCHS}, Early stopping: {EARLY_STOP}")
print(f"   Expected performance: P={TRIAL28_CONFIG['expected_performance']['precision']:.3f}, "
      f"R={TRIAL28_CONFIG['expected_performance']['recall']:.3f}")

start_time = datetime.now()

# Train model
history = model.fit(
    X_train_seq, y_train_seq,
    epochs=EPOCHS,
    batch_size=TRIAL28_CONFIG['batch'],
    validation_data=(X_val_seq, y_val_seq),
    callbacks=callbacks,
    verbose=2
)

training_time = datetime.now() - start_time
epochs_trained = len(history.history['loss'])

print(f"\n⏱️  Training completed in {training_time} ({epochs_trained} epochs)")

# ══════════════════════════════════════════════════════════════════════
# Evaluate Model
# ══════════════════════════════════════════════════════════════════════
print(f"\n📊 Evaluating model...")

y_pred_prob = model.predict(X_val_seq, verbose=0).ravel()
y_pred = (y_pred_prob >= DECISION_THRESHOLD).astype(int)

# Calculate metrics
precision = precision_score(y_val_seq, y_pred, zero_division=0)
recall = recall_score(y_val_seq, y_pred, zero_division=0)
f1 = f1_score(y_val_seq, y_pred, zero_division=0)
f05 = fbeta_score(y_val_seq, y_pred, beta=0.5, zero_division=0)
accuracy = accuracy_score(y_val_seq, y_pred)
auc = roc_auc_score(y_val_seq, y_pred_prob)

# Display results
print(f"\n📈 Final Results:")
expected = TRIAL28_CONFIG['expected_performance']
print(f"   Precision: {precision:.3f} (expected: {expected['precision']:.3f}) "
      f"[{precision - expected['precision']:+.3f}]")
print(f"   Recall:    {recall:.3f} (expected: {expected['recall']:.3f}) "
      f"[{recall - expected['recall']:+.3f}]")
print(f"   F1 Score:  {f1:.3f} (expected: {expected['f1']:.3f}) "
      f"[{f1 - expected['f1']:+.3f}]")
print(f"   F0.5 Score: {f05:.3f} (expected: {expected['f05']:.3f}) "
      f"[{f05 - expected['f05']:+.3f}]")
print(f"   Accuracy:  {accuracy:.3f}")
print(f"   AUC:       {auc:.3f}")
print(f"   Positive predictions: {np.sum(y_pred)}")

# Confusion matrix
cm = confusion_matrix(y_val_seq, y_pred)
print(f"\n📋 Confusion Matrix:")
print(cm)

print(f"\n📋 Classification Report:")
print(classification_report(y_val_seq, y_pred, target_names=["Down", "Up"]))

# ══════════════════════════════════════════════════════════════════════
# Generate Predictions CSV
# ══════════════════════════════════════════════════════════════════════
print(f"\n📁 Generating predictions CSV...")

# Get validation timestamps (accounting for window offset)
val_start_idx = split_idx + TRIAL28_CONFIG['window']
val_timestamps = df.index[val_start_idx:val_start_idx + len(y_pred_prob)]

# Calculate probabilities
prob_up = y_pred_prob
prob_down = 1.0 - prob_up
winning_prob = np.maximum(prob_up, prob_down)

# Create predictions dataframe
predictions_df = pd.DataFrame({
    'timestamp': val_timestamps.strftime('%Y-%m-%d %H:%M:%S'),
    'prob_up': prob_up,
    'prob_down': prob_down,
    'winning_prob': winning_prob,
    'prediction': y_pred,
    'actual': y_val_seq
})

# Save predictions CSV
predictions_df.to_csv(PREDICTIONS_OUT, index=False, float_format='%.6f')

print(f"   Predictions saved: {PREDICTIONS_OUT}")
print(f"   Total predictions: {len(predictions_df):,}")

# Show sample predictions
print(f"\n📋 Sample predictions:")
print(predictions_df.head(10).to_string(index=False))

# ══════════════════════════════════════════════════════════════════════
# Save Model and Summary
# ══════════════════════════════════════════════════════════════════════
print(f"\n💾 Saving model and summary...")

# Save model
model.save(MODEL_OUT)

# Create comprehensive summary
summary = {
    "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "model_type": "GRU_Trial28_BestOverall",
    "configuration": TRIAL28_CONFIG,
    "dataset_info": {
        "total_samples": len(df),
        "train_samples": len(X_train_scaled),
        "val_samples": len(X_val_scaled),
        "features": n_features,
        "window_size": TRIAL28_CONFIG['window'],
        "train_period": f"{df.index[0]} to {df.index[split_idx-1]}",
        "val_period": f"{df.index[split_idx]} to {df.index[-1]}"
    },
    "training_info": {
        "epochs_trained": epochs_trained,
        "training_time_seconds": training_time.total_seconds(),
        "early_stopping_patience": EARLY_STOP,
        "final_train_loss": float(history.history['loss'][-1]),
        "final_val_loss": float(history.history['val_loss'][-1]),
        "best_val_loss": float(min(history.history['val_loss']))
    },
    "performance_metrics": {
        "precision": float(precision),
        "recall": float(recall),
        "f1_score": float(f1),
        "f05_score": float(f05),
        "accuracy": float(accuracy),
        "auc": float(auc),
        "decision_threshold": DECISION_THRESHOLD,
        "positive_predictions": int(np.sum(y_pred)),
        "confusion_matrix": cm.tolist()
    },
    "expected_vs_actual": {
        "precision_diff": float(precision - expected['precision']),
        "recall_diff": float(recall - expected['recall']),
        "f1_diff": float(f1 - expected['f1']),
        "f05_diff": float(f05 - expected['f05'])
    },
    "class_distribution": {
        "train_positive_rate": float(np.mean(y_train)),
        "val_positive_rate": float(np.mean(y_val)),
        "train_counts": [int(np.sum(y_train == 0)), int(np.sum(y_train == 1))],
        "val_counts": [int(np.sum(y_val == 0)), int(np.sum(y_val == 1))]
    }
}

# Save summary
with open(SUMMARY_JSON, "w") as f:
    json.dump(summary, f, indent=2)

# ══════════════════════════════════════════════════════════════════════
# Final Report
# ══════════════════════════════════════════════════════════════════════
print(f"\n🎉 Trial 28 Final Training Complete!")
print(f"═" * 50)
print(f"📈 Performance Summary:")
print(f"   Precision: {precision:.3f} (target: 0.551)")
print(f"   Recall: {recall:.3f} (target: 0.437)")
print(f"   F1 Score: {f1:.3f} (target: 0.488)")
print(f"   F0.5 Score: {f05:.3f} (target: 0.524)")
print(f"   AUC: {auc:.3f}")

print(f"\n📁 Files Generated:")
print(f"   • {MODEL_OUT} - Trained GRU model")
print(f"   • {SCALER_OUT} - Feature scaler")
print(f"   • {PREDICTIONS_OUT} - Validation predictions ({len(predictions_df):,} rows)")
print(f"   • {SUMMARY_JSON} - Complete training summary")

print(f"\n🎯 Model ready for production trading!")
print(f"   Expected: ~55% precision with ~44% recall")
print(f"   Configuration: 36-window, 96-unit, 2-layer GRU")

# Cleanup
tf.keras.backend.clear_session()
gc.collect()

print(f"\n✨ Training pipeline completed successfully!")

⚠️  No GPU detected, using CPU
🚀 GRU Trial 28 Final Training
🎯 Configuration: BestOverall (0.551 precision, 0.437 recall)
📊 Loading and preprocessing data...
   Loaded data: (11476, 66)
   After cleaning: (11476, 20)
   Features: 19
   Target distribution: [5618 5858]
   Train samples: 9,180
   Val samples: 2,296
   Scaler saved: gru_trial28_scaler.pkl

🔄 Creating sequences (window=36)...
   Train sequences: 9,144
   Val sequences: 2,260

🏗️ Building GRU model...
   Units: 96, Layers: 2
   Dropout: 0.294, LR: 0.001390
   L2: 2.52e-04, Batch: 32

📋 Model architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                               ┃ Output Shape                    ┃           Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                                  │ ?                               │       0 (unbuilt) │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ gru_1 (GRU)                                │ ?                               │       0 (unbuilt) │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dropout (Dropout)                          │ ?                               │                 0 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense (Dense)                              │ ?                               │       0 (unbuilt) │
└────────────────────────────────────────────┴─────────────────────────────────┴───────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


🚀 Training model...
   Epochs: 80, Early stopping: 15
   Expected performance: P=0.551, R=0.437
Epoch 1/80
286/286 - 7s - 24ms/step - accuracy: 0.5024 - loss: 0.7586 - val_accuracy: 0.4996 - val_loss: 0.7292 - learning_rate: 0.0014
Epoch 2/80
286/286 - 4s - 15ms/step - accuracy: 0.5094 - loss: 0.7253 - val_accuracy: 0.4867 - val_loss: 0.7220 - learning_rate: 0.0014
Epoch 3/80
286/286 - 6s - 20ms/step - accuracy: 0.5046 - loss: 0.7160 - val_accuracy: 0.4796 - val_loss: 0.7143 - learning_rate: 0.0014
Epoch 4/80
286/286 - 9s - 30ms/step - accuracy: 0.5107 - loss: 0.7094 - val_accuracy: 0.4823 - val_loss: 0.7135 - learning_rate: 0.0014
Epoch 5/80
286/286 - 8s - 30ms/step - accuracy: 0.5086 - loss: 0.7052 - val_accuracy: 0.4996 - val_loss: 0.7036 - learning_rate: 0.0014
Epoch 6/80
286/286 - 8s - 30ms/step - accuracy: 0.5172 - loss: 0.7020 - val_accuracy: 0.4987 - val_loss: 0.7010 - learning_rate: 0.0014
Epoch 7/80
286/286 - 8s - 30ms/step - accuracy: 0.5197 - loss: 0.6986 - val_accuracy: 0


📈 Final Results:
   Precision: 0.536 (expected: 0.551) [-0.015]
   Recall:    0.516 (expected: 0.437) [+0.079]
   F1 Score:  0.526 (expected: 0.488) [+0.038]
   F0.5 Score: 0.532 (expected: 0.524) [+0.008]
   Accuracy:  0.516
   AUC:       0.527
   Positive predictions: 1130

📋 Confusion Matrix:
[[561 524]
 [569 606]]

📋 Classification Report:
              precision    recall  f1-score   support

        Down       0.50      0.52      0.51      1085
          Up       0.54      0.52      0.53      1175

    accuracy                           0.52      2260
   macro avg       0.52      0.52      0.52      2260
weighted avg       0.52      0.52      0.52      2260


📁 Generating predictions CSV...
   Predictions saved: gru_trial28_predictions.csv
   Total predictions: 2,260

📋 Sample predictions:
          timestamp  prob_up  prob_down  winning_prob  prediction  actual
2024-03-16 12:00:00 0.519279   0.480721      0.519279           1       1
2024-03-16 16:00:00 0.520203   0.479797     

In [6]:
"""
gru_trial28_final_training.py
─────────────────────────────────────────────────────────
Final training of Trial 28 (BestOverall) GRU configuration.
Trains the model and generates predictions CSV in the exact format requested.

Trial 28 Config: 0.551 precision, 0.437 recall, 0.488 F1, 0.524 F0.5
"""

import os
import json
import gc
import warnings
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score, f1_score, 
                             fbeta_score, accuracy_score, roc_auc_score,
                             confusion_matrix, classification_report)
import tensorflow as tf
from tensorflow import keras

# ══════════════════════════════════════════════════════════════════════
# Setup
# ══════════════════════════════════════════════════════════════════════
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Configure GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU configured: {len(gpus)} device(s)")
    except RuntimeError as e:
        print(f"⚠️  GPU configuration failed: {e}")
else:
    print("⚠️  No GPU detected, using CPU")

# ══════════════════════════════════════════════════════════════════════
# Configuration
# ══════════════════════════════════════════════════════════════════════
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

VAL_FRAC = 0.20
DECISION_THRESHOLD = 0.5
EPOCHS = 80
EARLY_STOP = 15

# Output files
MODEL_OUT = "gru_trial28_final.h5"
SCALER_OUT = "gru_trial28_scaler.pkl"
PREDICTIONS_OUT = "gru_trial28_predictions.csv"
SUMMARY_JSON = "gru_trial28_training_summary.json"

# Complete drop columns list (same as optimization)
DROP_COLS = [
    'open', 'high', 'low', 'close',
    'typical_price', 'vwap_24h', 'close_4h',
    'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'bollinger_upper', 'bollinger_lower', 'bollinger_width',
    'resistance_level', 'support_level',
    'high_low', 'high_close', 'low_close', 'true_range',
    'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'CCI', 'stoch_%D', 'parkinson_vol',
    'ema_cross_down', 'macd_cross_down', 'ema_cross_up', 'macd_cross_up',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'trending_market',
    'oversold_reversal', 'overbought_reversal',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'timestamp', 'date', 'Unnamed: 0'
]

# Trial 28 Optimal Configuration
TRIAL28_CONFIG =     {
        'name': 'Trial_4_Balanced',
        'window': 30, 'units': 128, 'layers': 1, 'dropout': 0.3473773873201034,
        'lr': 0.0019947718789028682, 'l2': 2.497073714505272e-05, 'batch': 64,
        'expected_performance': {'precision': 0.547, 'recall': 0.424, 'f1': 0.478, 'f05': 0.517}
    }

# ══════════════════════════════════════════════════════════════════════
# Helper Functions
# ══════════════════════════════════════════════════════════════════════
def create_sequences(data, labels, window_size):
    """Create sequences for time series data."""
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(labels[i])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int8)

def build_gru_model(config, n_features):
    """Build GRU model with Trial 28 configuration."""
    model = keras.Sequential()
    
    for i in range(config['layers']):
        return_sequences = (i < config['layers'] - 1)
        model.add(keras.layers.GRU(
            config['units'],
            return_sequences=return_sequences,
            dropout=config['dropout'],
            recurrent_dropout=config['dropout'] * 0.6,
            kernel_regularizer=keras.regularizers.l2(config['l2']),
            recurrent_regularizer=keras.regularizers.l2(config['l2'])
        ))
    
    model.add(keras.layers.Dropout(config['dropout']))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config['lr']),
        loss="binary_crossentropy",
        metrics=['accuracy']
    )
    
    return model

# ══════════════════════════════════════════════════════════════════════
# Data Loading and Preprocessing
# ══════════════════════════════════════════════════════════════════════
print("🚀 GRU Trial 28 Final Training")
print("=" * 45)
print("🎯 Configuration: BestOverall (0.551 precision, 0.437 recall)")
print("📊 Loading and preprocessing data...")

# Load data
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df[df.index >= "2020-01-01"]  # Same as optimization
print(f"   Loaded data: {df.shape}")

# Clean data
if 'target' not in df.columns:
    raise ValueError("❌ Target column not found!")

# Drop columns
cols_to_drop = [c for c in DROP_COLS if c in df.columns]
df = df.drop(columns=cols_to_drop)
df = df[df["target"].notna()].dropna()
print(f"   After cleaning: {df.shape}")

# Prepare features and target
features = df.drop(columns="target")
target = df["target"].astype(int).values
n_features = features.shape[1]

print(f"   Features: {n_features}")
print(f"   Target distribution: {np.bincount(target)}")

# Scale features
split_idx = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler()
scaler.fit(features.iloc[:split_idx])

X_train_scaled = scaler.transform(features.iloc[:split_idx])
X_val_scaled = scaler.transform(features.iloc[split_idx:])
y_train = target[:split_idx]
y_val = target[split_idx:]

print(f"   Train samples: {len(X_train_scaled):,}")
print(f"   Val samples: {len(X_val_scaled):,}")

# Save scaler
joblib.dump(scaler, SCALER_OUT)
print(f"   Scaler saved: {SCALER_OUT}")

# ══════════════════════════════════════════════════════════════════════
# Create Sequences
# ══════════════════════════════════════════════════════════════════════
print(f"\n🔄 Creating sequences (window={TRIAL28_CONFIG['window']})...")

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, TRIAL28_CONFIG['window'])
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val, TRIAL28_CONFIG['window'])

print(f"   Train sequences: {len(X_train_seq):,}")
print(f"   Val sequences: {len(X_val_seq):,}")

# ══════════════════════════════════════════════════════════════════════
# Build and Train Model
# ══════════════════════════════════════════════════════════════════════
print(f"\n🏗️ Building GRU model...")
print(f"   Units: {TRIAL28_CONFIG['units']}, Layers: {TRIAL28_CONFIG['layers']}")
print(f"   Dropout: {TRIAL28_CONFIG['dropout']:.3f}, LR: {TRIAL28_CONFIG['lr']:.6f}")
print(f"   L2: {TRIAL28_CONFIG['l2']:.2e}, Batch: {TRIAL28_CONFIG['batch']}")

tf.keras.backend.clear_session()
gc.collect()

model = build_gru_model(TRIAL28_CONFIG, n_features)

# Display model summary
print(f"\n📋 Model architecture:")
model.summary(line_length=100)

# Prepare callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=EARLY_STOP,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    )
]

print(f"\n🚀 Training model...")
print(f"   Epochs: {EPOCHS}, Early stopping: {EARLY_STOP}")
print(f"   Expected performance: P={TRIAL28_CONFIG['expected_performance']['precision']:.3f}, "
      f"R={TRIAL28_CONFIG['expected_performance']['recall']:.3f}")

start_time = datetime.now()

# Train model
history = model.fit(
    X_train_seq, y_train_seq,
    epochs=EPOCHS,
    batch_size=TRIAL28_CONFIG['batch'],
    validation_data=(X_val_seq, y_val_seq),
    callbacks=callbacks,
    verbose=2
)

training_time = datetime.now() - start_time
epochs_trained = len(history.history['loss'])

print(f"\n⏱️  Training completed in {training_time} ({epochs_trained} epochs)")

# ══════════════════════════════════════════════════════════════════════
# Evaluate Model
# ══════════════════════════════════════════════════════════════════════
print(f"\n📊 Evaluating model...")

y_pred_prob = model.predict(X_val_seq, verbose=0).ravel()
y_pred = (y_pred_prob >= DECISION_THRESHOLD).astype(int)

# Calculate metrics
precision = precision_score(y_val_seq, y_pred, zero_division=0)
recall = recall_score(y_val_seq, y_pred, zero_division=0)
f1 = f1_score(y_val_seq, y_pred, zero_division=0)
f05 = fbeta_score(y_val_seq, y_pred, beta=0.5, zero_division=0)
accuracy = accuracy_score(y_val_seq, y_pred)
auc = roc_auc_score(y_val_seq, y_pred_prob)

# Display results
print(f"\n📈 Final Results:")
expected = TRIAL28_CONFIG['expected_performance']
print(f"   Precision: {precision:.3f} (expected: {expected['precision']:.3f}) "
      f"[{precision - expected['precision']:+.3f}]")
print(f"   Recall:    {recall:.3f} (expected: {expected['recall']:.3f}) "
      f"[{recall - expected['recall']:+.3f}]")
print(f"   F1 Score:  {f1:.3f} (expected: {expected['f1']:.3f}) "
      f"[{f1 - expected['f1']:+.3f}]")
print(f"   F0.5 Score: {f05:.3f} (expected: {expected['f05']:.3f}) "
      f"[{f05 - expected['f05']:+.3f}]")
print(f"   Accuracy:  {accuracy:.3f}")
print(f"   AUC:       {auc:.3f}")
print(f"   Positive predictions: {np.sum(y_pred)}")

# Confusion matrix
cm = confusion_matrix(y_val_seq, y_pred)
print(f"\n📋 Confusion Matrix:")
print(cm)

print(f"\n📋 Classification Report:")
print(classification_report(y_val_seq, y_pred, target_names=["Down", "Up"]))

# ══════════════════════════════════════════════════════════════════════
# Generate Predictions CSV
# ══════════════════════════════════════════════════════════════════════
print(f"\n📁 Generating predictions CSV...")

# Get validation timestamps (accounting for window offset)
val_start_idx = split_idx + TRIAL28_CONFIG['window']
val_timestamps = df.index[val_start_idx:val_start_idx + len(y_pred_prob)]

# Calculate probabilities
prob_up = y_pred_prob
prob_down = 1.0 - prob_up
winning_prob = np.maximum(prob_up, prob_down)

# Create predictions dataframe
predictions_df = pd.DataFrame({
    'timestamp': val_timestamps.strftime('%Y-%m-%d %H:%M:%S'),
    'prob_up': prob_up,
    'prob_down': prob_down,
    'winning_prob': winning_prob,
    'prediction': y_pred,
    'actual': y_val_seq
})

# Save predictions CSV
predictions_df.to_csv(PREDICTIONS_OUT, index=False, float_format='%.6f')

print(f"   Predictions saved: {PREDICTIONS_OUT}")
print(f"   Total predictions: {len(predictions_df):,}")

# Show sample predictions
print(f"\n📋 Sample predictions:")
print(predictions_df.head(10).to_string(index=False))

# ══════════════════════════════════════════════════════════════════════
# Save Model and Summary
# ══════════════════════════════════════════════════════════════════════
print(f"\n💾 Saving model and summary...")

# Save model
model.save(MODEL_OUT)

# Create comprehensive summary
summary = {
    "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "model_type": "GRU_Trial28_BestOverall",
    "configuration": TRIAL28_CONFIG,
    "dataset_info": {
        "total_samples": len(df),
        "train_samples": len(X_train_scaled),
        "val_samples": len(X_val_scaled),
        "features": n_features,
        "window_size": TRIAL28_CONFIG['window'],
        "train_period": f"{df.index[0]} to {df.index[split_idx-1]}",
        "val_period": f"{df.index[split_idx]} to {df.index[-1]}"
    },
    "training_info": {
        "epochs_trained": epochs_trained,
        "training_time_seconds": training_time.total_seconds(),
        "early_stopping_patience": EARLY_STOP,
        "final_train_loss": float(history.history['loss'][-1]),
        "final_val_loss": float(history.history['val_loss'][-1]),
        "best_val_loss": float(min(history.history['val_loss']))
    },
    "performance_metrics": {
        "precision": float(precision),
        "recall": float(recall),
        "f1_score": float(f1),
        "f05_score": float(f05),
        "accuracy": float(accuracy),
        "auc": float(auc),
        "decision_threshold": DECISION_THRESHOLD,
        "positive_predictions": int(np.sum(y_pred)),
        "confusion_matrix": cm.tolist()
    },
    "expected_vs_actual": {
        "precision_diff": float(precision - expected['precision']),
        "recall_diff": float(recall - expected['recall']),
        "f1_diff": float(f1 - expected['f1']),
        "f05_diff": float(f05 - expected['f05'])
    },
    "class_distribution": {
        "train_positive_rate": float(np.mean(y_train)),
        "val_positive_rate": float(np.mean(y_val)),
        "train_counts": [int(np.sum(y_train == 0)), int(np.sum(y_train == 1))],
        "val_counts": [int(np.sum(y_val == 0)), int(np.sum(y_val == 1))]
    }
}

# Save summary
with open(SUMMARY_JSON, "w") as f:
    json.dump(summary, f, indent=2)

# ══════════════════════════════════════════════════════════════════════
# Final Report
# ══════════════════════════════════════════════════════════════════════
print(f"\n🎉 Trial 28 Final Training Complete!")
print(f"═" * 50)
print(f"📈 Performance Summary:")
print(f"   Precision: {precision:.3f} (target: 0.551)")
print(f"   Recall: {recall:.3f} (target: 0.437)")
print(f"   F1 Score: {f1:.3f} (target: 0.488)")
print(f"   F0.5 Score: {f05:.3f} (target: 0.524)")
print(f"   AUC: {auc:.3f}")

print(f"\n📁 Files Generated:")
print(f"   • {MODEL_OUT} - Trained GRU model")
print(f"   • {SCALER_OUT} - Feature scaler")
print(f"   • {PREDICTIONS_OUT} - Validation predictions ({len(predictions_df):,} rows)")
print(f"   • {SUMMARY_JSON} - Complete training summary")

print(f"\n🎯 Model ready for production trading!")
print(f"   Expected: ~55% precision with ~44% recall")
print(f"   Configuration: 36-window, 96-unit, 2-layer GRU")

# Cleanup
tf.keras.backend.clear_session()
gc.collect()

print(f"\n✨ Training pipeline completed successfully!")

⚠️  No GPU detected, using CPU
🚀 GRU Trial 28 Final Training
🎯 Configuration: BestOverall (0.551 precision, 0.437 recall)
📊 Loading and preprocessing data...
   Loaded data: (11476, 66)
   After cleaning: (11476, 20)
   Features: 19
   Target distribution: [5618 5858]
   Train samples: 9,180
   Val samples: 2,296
   Scaler saved: gru_trial28_scaler.pkl

🔄 Creating sequences (window=30)...
   Train sequences: 9,150
   Val sequences: 2,266

🏗️ Building GRU model...
   Units: 128, Layers: 1
   Dropout: 0.347, LR: 0.001995
   L2: 2.50e-05, Batch: 64

📋 Model architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                               ┃ Output Shape                    ┃           Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                                  │ ?                               │       0 (unbuilt) │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dropout (Dropout)                          │ ?                               │                 0 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense (Dense)                              │ ?                               │       0 (unbuilt) │
└────────────────────────────────────────────┴─────────────────────────────────┴───────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


🚀 Training model...
   Epochs: 80, Early stopping: 15
   Expected performance: P=0.547, R=0.424
Epoch 1/80
143/143 - 3s - 20ms/step - accuracy: 0.5014 - loss: 0.7136 - val_accuracy: 0.4965 - val_loss: 0.7100 - learning_rate: 0.0020
Epoch 2/80
143/143 - 1s - 9ms/step - accuracy: 0.5014 - loss: 0.7054 - val_accuracy: 0.4943 - val_loss: 0.7075 - learning_rate: 0.0020
Epoch 3/80
143/143 - 1s - 9ms/step - accuracy: 0.5082 - loss: 0.6999 - val_accuracy: 0.5132 - val_loss: 0.7022 - learning_rate: 0.0020
Epoch 4/80
143/143 - 1s - 9ms/step - accuracy: 0.5153 - loss: 0.6976 - val_accuracy: 0.5035 - val_loss: 0.7038 - learning_rate: 0.0020
Epoch 5/80
143/143 - 1s - 9ms/step - accuracy: 0.5217 - loss: 0.6962 - val_accuracy: 0.4996 - val_loss: 0.7028 - learning_rate: 0.0020
Epoch 6/80
143/143 - 1s - 9ms/step - accuracy: 0.5144 - loss: 0.6969 - val_accuracy: 0.4978 - val_loss: 0.7039 - learning_rate: 0.0020
Epoch 7/80
143/143 - 1s - 10ms/step - accuracy: 0.5252 - loss: 0.6954 - val_accuracy: 0.5000


📈 Final Results:
   Precision: 0.555 (expected: 0.547) [+0.008]
   Recall:    0.359 (expected: 0.424) [-0.065]
   F1 Score:  0.436 (expected: 0.478) [-0.042]
   F0.5 Score: 0.500 (expected: 0.517) [-0.017]
   Accuracy:  0.518
   AUC:       0.532
   Positive predictions: 760

📋 Confusion Matrix:
[[751 338]
 [755 422]]

📋 Classification Report:
              precision    recall  f1-score   support

        Down       0.50      0.69      0.58      1089
          Up       0.56      0.36      0.44      1177

    accuracy                           0.52      2266
   macro avg       0.53      0.52      0.51      2266
weighted avg       0.53      0.52      0.50      2266


📁 Generating predictions CSV...
   Predictions saved: gru_trial28_predictions.csv
   Total predictions: 2,266

📋 Sample predictions:
          timestamp  prob_up  prob_down  winning_prob  prediction  actual
2024-03-15 12:00:00 0.593111   0.406889      0.593111           1       1
2024-03-15 16:00:00 0.631037   0.368963      

In [ ]:
#"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\gru_trial28_predictions.csv"



In [7]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the predictions CSV
csv_path = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\gru_trial28_f05_preds.csv"
df = pd.read_csv(csv_path)

# Ensure column names are correct and lowercase
df.columns = df.columns.str.strip().str.lower()

# Extract actual and predicted values
y_true = df['actual']
y_pred = df['prediction']  # prediction at threshold 0.5

# Calculate metrics
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Print results
print("📊 Evaluation at threshold 0.5:")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1 Score : {f1:.3f}")


📊 Evaluation at threshold 0.5:
Precision: 0.528
Recall   : 0.951
F1 Score : 0.679
